In [7]:
import pandas as pd
import re

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result


df = pd.read_csv("review_data.csv")
df["ko_text"] = df["review"].apply(lambda x: text_cleaning(x))

df = df[df['ko_text'].str.len() > 0]
df.head()

,score,review,y,ko_text
1,4,녹는다 녹아,1,녹는다 녹아
2,4,좋아요,1,좋아요
3,5,흠잡을곳 없는 반찬이 정갈합니다,1,흠잡을곳 없는 반찬이 정갈합니다
4,5,코로나 땜에 걱정했는데 방역수칙도 잘 지키시고 살치실이랑 등심 부드러워서 너무 좋아...,1,코로나 땜에 걱정했는데 방역수칙도 잘 지키시고 살치실이랑 등심 부드러워서 너무 좋아...
5,5,살치살 미쳤네요.. 대박입니다 퀄리티 굳이에요~,1,살치살 미쳤네요 대박입니다 퀄리티 굳이에요


In [27]:
from konlpy.tag import Okt

# konlpy라이브러리로 텍스트 데이터에서 형태소를 추출합니다.
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

result = get_pos(df['ko_text'].values[0])
print(result)
result_2 = get_pos(df['ko_text'].values[1])
print(result_2)

['녹는다/Verb', '녹아/Verb']
['좋아요/Adjective']


In [36]:
'''
코딩 목표
pickle모듈을 사용하여 형태소 단위로 분석한 리뷰들을 저장한다. 
(리스트 단위로 저장하는것이 좋을듯하다)
'''
import pickle

col_to_list = df['ko_text'].tolist()

with open("parsed_review_data.pkl", "wb") as f:
    for i in col_to_list:
        tmp = get_pos(i) 
        #def get_pos(text: str): list
        pickle.dump(tmp, f)


num_of_file = 0


with open("parsed_review_data.pkl", "rb") as f:
    try:
        while True:
            value = pickle.load(f)
            num_of_file += 1
            print(num_of_file, "번째: ", value)
    except EOFError:
        print("파일 읽기 종료")

1 번째:  ['녹는다/Verb', '녹아/Verb']
2 번째:  ['좋아요/Adjective']
3 번째:  ['흠잡을/Adjective', '곳/Noun', '없는/Adjective', '반찬/Noun', '이/Josa', '정갈합니다/Adjective']
4 번째:  ['코로나/Noun', '땜/Noun', '에/Josa', '걱정/Noun', '했는데/Verb', '방역/Noun', '수/Modifier', '칙/Noun', '도/Josa', '잘/Verb', '지키시고/Verb', '살치실이랑/Verb', '등심/Noun', '부드러워서/Adjective', '너무/Adverb', '좋아서/Adjective', '댓글/Noun', '남깁니다/Verb']
5 번째:  ['살치/Verb', '살/Noun', '미쳤네요/Adjective', '대박/Noun', '입니다/Adjective', '퀄리티/Noun', '굳이/Noun', '에요/Josa']
6 번째:  ['최고/Noun']
7 번째:  ['외식/Noun', '하기에/Verb', '딱/Adverb', '좋습니다/Adjective', '격/Noun', '자체/Noun', '가/Josa', '다른/Noun', '음식/Noun', '의/Josa', '품격/Noun', '서비스/Noun', '마인드/Noun', '뭐/Noun', '하나/Noun', '흠잡을/Adjective', '곳/Noun', '이/Josa', '없는/Adjective', '정말/Noun', '멋진/Adjective', '식당/Noun', '이네/Josa', '요/Noun', '자주/Noun', '이/Suffix', '용하겠습니다/Adjective']
8 번째:  ['한우/Noun', '살이/Noun', '아주/Noun', '그냥/Noun', '입/Noun', '에서/Josa', '녹아요/Verb', '한번/Noun', '왔는데/Verb', '또/Noun', '오고싶은/Verb', '맛/Noun', '입니다/Adjective']
9 번

In [ ]:
'''
코딩목표
corpus(말뭉치, 형태소) index 생성하기
말뭉치를 벡터화한다 (0과 1로 표현한다)
'''

from sklearn.feature_extraction.text import CountVectorizer

index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist())

In [46]:
'''
코딩목표
CounterVectorize.fit_transform()을 이해해보자
'''
from sklearn.feature_extraction.text import CountVectorizer

test_ko_list = ["맛있어요", "맛없어요", "사장님이 친절해요", "매장이 넓고 깨끗해요", "그냥 그래요 시설은 좋아요"]

index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x))
_X = index_vectorizer.fit_transform(test_ko_list)

print(index_vectorizer.get_feature_names_out())

print(type(index_vectorizer.get_feature_names_out()))

print(_X.toarray())

['그냥/Noun' '그래요/Adjective' '깨끗해요/Adjective' '넓고/Adjective' '님/Suffix'
 '맛없어요/Adjective' '맛있어요/Adjective' '매장/Noun' '사장/Noun' '시설/Noun' '은/Josa'
 '이/Josa' '좋아요/Adjective' '친절해요/Adjective']
<class 'numpy.ndarray'>
[[0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 1 0 1]
 [0 0 1 1 0 0 0 1 0 0 0 1 0 0]
 [1 1 0 0 0 0 0 0 0 1 1 0 1 0]]


tf-idf feature의 행렬로 바꿔야한다 (370, x)

https://wikidocs.net/31698

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html